In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import regex as re
from jellyfish import jaro_similarity
from tqdm import tqdm

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep

In [ ]:
filename = "covid_philippines"
path = "datasets/" + filename + "/" + filename + ".csv"
video_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)

In [ ]:
video_df.head()

In [ ]:
video_id_list = video_df["video_id"].tolist()

In [ ]:
for video_id in video_id_list[0:5]:
    print(f"https://www.youtube.com/watch?v={video_id}")

---

In [ ]:
driver = webdriver.Firefox()
driver.get("https://www.thehoth.com/backlinks-checker/")
driver.maximize_window()

In [ ]:
# Wait for page to load and locate textbox
WebDriverWait(driver, 300).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "hothtools"))
    )

# Wait for textbox to be clickable and then click
WebDriverWait(driver, 300).until(
        EC.element_to_be_clickable((By.ID, "targeturl"))
    )

# Input video link
targeturl = driver.find_element(By.ID, "targeturl")
targeturl.click()
targeturl.send_keys("https://www.youtube.com/watch?v=aLZ85hb4wjE")

# Select 'This Exact URL'
dropdown = driver.find_element(By.ID, "mode")
dropdown.click()
option = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/select/option[3]")
option.click()

# Submit
submit = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/button")
submit.click()

In [ ]:
# In case of Captcha page
iframes = driver.find_elements(By.TAG_NAME, "iframe")

for iframe in iframes:
    try:
        driver.switch_to.frame(iframe)
        checkbox = driver.find_element(By.ID, "recaptcha-anchor")
        checkbox.click()
        break
    except:
        print("Not Found")
        driver.switch_to.default_content()

sleep(15)
        
driver.switch_to.default_content()
driver.switch_to.frame("hothtools")
submit = driver.find_element(
    By.XPATH, "//*[@id='submit']"
)
submit.click()

In [ ]:
# Get external backlinks
driver.switch_to.default_content()
driver.switch_to.frame("hothtools")
# 
WebDriverWait(driver, 300).until(
        EC.visibility_of_element_located(
            (By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]")
        )
    )
result = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]")
print(result.text)

---

In [ ]:
def get_backlinks(video_id_list):
    backlinks = {}
    
    driver = webdriver.Firefox()
    driver.get("https://www.thehoth.com/backlinks-checker/")
    driver.maximize_window()
    
    captcha_passed = False
    
    
    try:
        for video_id in video_id_list:
            # print("Inputting video link")
            # Wait for page to load and locate textbox
            # print("Waiting for frame")
            WebDriverWait(driver, 300).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "hothtools"))
                )

            # Wait for textbox to be clickable and then click
            # print("Waiting for textbox")
            WebDriverWait(driver, 300).until(
                    EC.element_to_be_clickable((By.ID, "targeturl"))
                )

            # Input video link
            targeturl = driver.find_element(By.ID, "targeturl")
            targeturl.click()
            targeturl.send_keys(Keys.CONTROL + "A")
            targeturl.send_keys(Keys.BACKSPACE)
            targeturl.send_keys(f"https://www.youtube.com/watch?v={video_id}")

            # Select 'This Exact URL'
            dropdown = driver.find_element(By.ID, "mode")
            dropdown.click()
            option = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/select/option[3]")
            option.click()

            # Submit
            submit = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/button")
            submit.click()

            sleep(5)

            try:
                if not captcha_passed:
                    # print("Solving CAPTCHA")
                    # In case of Captcha page
                    iframes = driver.find_elements(By.TAG_NAME, "iframe")
                    cont = False
                    for iframe in iframes:
                        try:
                            driver.switch_to.frame(iframe)
                            checkbox = driver.find_element(By.ID, "recaptcha-anchor")
                            checkbox.click()
                            cont = True
                            break
                        except:
                            # print("Not Found")
                            driver.switch_to.default_content()

                    if not cont:
                        raise Exception("No CAPTCHA iframe found.")

                    # Giving enough time to manually solve CAPTCHA puzzle
                    sleep(15)

                    driver.switch_to.default_content()
                    driver.switch_to.frame("hothtools")
                    submit = driver.find_element(
                        By.XPATH, "//*[@id='submit']"
                    )
                    submit.click()
                    captcha_passed = True
                    sleep(3)
            except:
                pass
            finally:
                # print("Getting results")
                # Get external backlinks
                driver.switch_to.default_content()
                driver.switch_to.frame("hothtools")
                try:
                    WebDriverWait(driver, 300).until(
                        EC.element_to_be_clickable((By.ID, "targeturl"))
                    )
                    
                    result = driver.find_element(
                        By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]"
                    ).text
                except:
                    result = 0
                finally:
                    backlinks[video_id] = result
                    driver.switch_to.default_content()
    except:
        print("Error raise by Selenium Webdriver. Ending process and saving data.")
    # '''
    finally:
        driver.close()
    # '''
    
    return backlinks

In [ ]:
backlinks = get_backlinks(video_id_list)

In [ ]:
print(f"Completed: {len(backlinks)}/{len(video_id_list)}")